In [1]:
import numpy as np

In [2]:
# Loading all the datasets

X_seen=np.load('X_seen.npy',encoding='bytes',allow_pickle=True) #	(40 x N_i x D): Seen training class

Attr=np.load('class_attributes_seen.npy',encoding='bytes',allow_pickle=True) # (40 x 85): Seen class attributes

UAttr=np.load('class_attributes_unseen.npy',encoding='bytes',allow_pickle=True) # (10 x 85): Unsee class atrributes

Xtest=np.load('Xtest.npy',encoding='bytes',allow_pickle=True)
Ytest=np.load('Ytest.npy',encoding='bytes',allow_pickle=True)

In [3]:
# Calculating mean of the each of these classes

Mean = np.zeros((40,4096)) #Mean will store all the 40 means


for i in range(40):
    Mean[i] = np.mean(X_seen[i],axis=0)

print("Shape of Mean is ",Mean.shape)
# printing shape for cross validation

Shape of Mean is  (40, 4096)


In [4]:
# ensuring float dtype
Mean = Mean.astype(float) # (40 x 4096) matrix containg the means of classes
Attr = Attr.astype(float)  # (40 x 85) matrix containing class attributes

### **Regression Model for Mean prediction**

for given the attribute matrix $A_s$ and mean matrix $M_s$  
we need to calculate the weight matrix $W$ of shape $85 \times 4096$
 
$$ W = (A_s^TA_s + \lambda I)^{-1}A_s^T M_s $$

So, I will first calculate $W_{left}$ which is equal to $(A_s^TA_s + \lambda I)^{-1}$  
then I will calculate $W_{right}$ which is $A_s^T M_s$


In [5]:
# picking a random lambda for now
lamda = 0.1

W_l = np.matmul(Attr.T,Attr) + lamda*np.eye(85,85) #I need the inverse of this
W_l = np.linalg.inv(W_l)

W_r = np.matmul(Attr.T,Mean)

W = np.matmul(W_l,W_r)

print("Shape of W is ",W.shape)

Shape of W is  (85, 4096)


### **Predicting the mean of Unseen Classes**

To get mean matrix of unsee classes we have,
<boxed>
$$ M_{unseen} = A_{unseen}W$$
</boxed>


In [6]:
UMean = np.matmul(UAttr,W)
print("Shape of UMean is ",UMean.shape)

Shape of UMean is  (10, 4096)


### **Testing time**

for each test input $x_*$ calculate $\lVert x_*-\mu_{c} \rVert^2$ for each unseen class $c$

In [7]:
N = Xtest.shape[0] # number of testing inputs

D = np.ones((N,10),dtype=float) #this will have the all the corresponsing norms

for i in range(N):
    xi = Xtest[i]
    for uc in range(UMean.shape[0]):
        D[i][uc] = np.linalg.norm(xi-UMean[uc])

print("shape of D is",D.shape)

shape of D is (6180, 10)


In [8]:
# getting the predictions
Ypredicted = np.argmin(D,axis = 1) + 1 #adding to make it 1 based indexing

print("shape of Ypredicted is ",Ypredicted.shape)

shape of Ypredicted is  (6180,)


In [9]:
# reshaping the array of true values
Ytrue = Ytest.reshape(-1,)

# calculating the correct prediction
equals = (Ypredicted == Ytrue)
Corrects = np.sum(equals)

print(f"Accuracy of the model with lambda = {lamda} is {Corrects*100/N} %")

Accuracy of the model with lambda = 0.1 is 59.54692556634304 %


### **Checking predictions of all the $\lambda$'s**

In [11]:
lamda_set = [0.01, 0.1, 1, 10, 20, 50, 100]

for lamda in lamda_set:
    # calculate W
    W_l = np.matmul(Attr.T,Attr) + lamda*np.eye(85,85)

    W_l = np.linalg.inv(W_l)

    W_r = np.matmul(Attr.T,Mean)

    W = np.matmul(W_l,W_r)
    # ------------------
    # Get the Unseen_means
    UMean = np.matmul(UAttr,W)

    # generate the predictions

    N = Xtest.shape[0]

    D = np.ones((N,10),dtype=float)

    for i in range(N):
        xi = Xtest[i]
        for uc in range(UMean.shape[0]):
            D[i][uc] = np.linalg.norm(xi-UMean[uc])

    # making predictions and accuracies

    Ypredicted = np.argmin(D,axis = 1) + 1

    equal = (Ypredicted == Ytrue)
    C = np.sum(equal)

    print(f"Accuracy obtained for lambda = {lamda} is {C*100/N} %")

Accuracy obtained for lambda = 0.01 is 58.090614886731395 %
Accuracy obtained for lambda = 0.1 is 59.54692556634304 %
Accuracy obtained for lambda = 1 is 67.3948220064725 %
Accuracy obtained for lambda = 10 is 73.28478964401295 %
Accuracy obtained for lambda = 20 is 71.68284789644012 %
Accuracy obtained for lambda = 50 is 65.08090614886731 %
Accuracy obtained for lambda = 100 is 56.47249190938511 %
